In [2]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
from time import time
#from scipy.integrate import quad

###################################################################################################################
##パラメータの調整
N, V, t, mu, n_search, n_scf, error, check_gap =500, 2, 1, 0, 10, 100, 1e-10, 1e-6    # 7.525 #9.21
kBTs = np.linspace(0.18,0.3,2)
q  = 0

###################################################################################################################
## gap_eq をdef

def e_k_spin(k1, k2, q, y, B): 
    return 2*t*(np.cos((k1+(q/2)*np.pi))+np.cos((k2))) - mu 
    #return 2*t*(np.cos((k1+(q/2)))+np.cos((k2))) - mu + y * 1/2 * gu * B

def e_k_s(k1, k2, q, B):
    return (e_k_spin(k1, k2, q, 1, B) + e_k_spin(-1*k1, k2, q, -1, B))/2

def e_k_a(k1, k2, q, B):
    return (e_k_spin(k1, k2, q, 1, B) - e_k_spin(-1*k1, k2, q, -1, B))/2

def E_k_q(k1, k2, gap, q, B):
    return np.sqrt(e_k_s(k1, k2, q, B)**2 + gap**2)

def E_k_q_s(k1, k2, gap, q, y, B):
    return E_k_q(k1, k2, gap, q, B) + y * e_k_a(k1, k2, q, B)

def Fermi(beta, E):
    #return  1 / (np.exp(beta*E) + 1 )
    return (1 - np.tanh(beta*E/2)) /2

def func(k1, k2, gap, q, B): 
    return gap*(1-Fermi(beta, E_k_q_s(k1, k2, gap, q, -1, B))-Fermi(beta, E_k_q_s(k1, k2, gap, q, 1, B)))/(2*E_k_q(k1, k2, gap, q, B))

def rhs(gap, q, B):
    k1 = -1 * np.pi + 2 * arange(N) * np.pi / (N)
    kx, ky = meshgrid(k1, k1, indexing='ij')
    f = func(kx, ky, gap, q, B)
    return (V / (N**2)) * sum(f)

###################################################################################################################
#search_TC
time_binary_search_start = time()

kBT_a = kBTs[0]
kBT_b = kBTs[1]

ans1 = []
for j in range(n_search):
    kBT = (kBT_a+kBT_b)/2
    beta, d0 = 1/kBT, 100.0
    for k in range(1000): # 収束するまで最大1000回ループ
        d1 = rhs(d0, q, 0) 
        if abs(d1-d0) < error: break # 収束チェック
        d0 = d1
    ans1.append([kBT, d0, abs(d1-d0), k])
    if d0 < check_gap: 
        kBT_b =  (kBT_a + kBT_b) /2
    if d0 > check_gap: 
        kBT_a =  (kBT_a + kBT_b) /2   

time_binary_search_finish = time()

file = open("./output/kBT_gap_error_iter" + "_N_" + str(N) + "_V_" + str(V) + "_mu_" + str(mu) + "_t_" + str(t) + "_check_gap_" + str(check_gap) + "_kBT_[" + str(kBTs[0]) + "," + str(kBTs[-1]) + "]_n_serach_" + str(n_search) + "_q_" + str(q)  ,"w") 
for i in range(n_search):
        file.write(str(ans1[i][0]) + " " + str(ans1[i][1]) + " " + str(ans1[i][2]) + " " + str(ans1[i][3]) + " " +  "\n")
file.close()

time_binary_search = time_binary_search_finish - time_binary_search_start
print("time_binary_search : " + str(time_binary_search//3600) + "時間" + str((time_binary_search%3600)//60) +"分" + str(time_binary_search%60) + "秒")


KeyboardInterrupt: 